In [199]:
from bs4 import BeautifulSoup
import os
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time


In [200]:
url = 'https://supermarket-scrape-website.vercel.app/server/'

In [201]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options)
driver.get(url)
wait = WebDriverWait(driver, 10)  # Waits for up to 10 seconds

In [202]:
categories = driver.find_elements(By.CLASS_NAME, 'category')

In [203]:
folder_path = './csv_dumps/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [204]:
def extract_products (soup):
    products_list = pd.DataFrame()
    products = soup.find_all(class_='product')
    for product in products:
        name = product.find(class_='product_name').text
        price = product.find(class_='product_price').text
        products_list = pd.concat([products_list, pd.DataFrame([{'name' : name.replace('\n', ' ').replace('\r', ''), 'price' : price}])])
    return products_list

In [205]:
def get_select (driver):
    select = Select(driver.find_element(By.CLASS_NAME, 'page_dropdown'))
    return select

In [206]:
for category in categories:
    try :
        category.click()
        all_products = pd.DataFrame()
        select = get_select(driver)
        print (f'Extracting Category {category.text}')
        for option in select.options:
            select.select_by_visible_text(option.text)
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'page_dropdown')))
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            all_products = pd.concat([all_products, extract_products(soup)])
            time.sleep(random.uniform(1, 5))
        all_products.to_csv(f'csv_dumps/{category.text}.csv', index=False)
    except: 
        None   
    

Extracting Category Confectionery
Extracting Category Delicatessen
Extracting Category Household


In [194]:
if not os.path.exists('csv_final'):
    os.makedirs('csv_final')

final_file = pd.DataFrame()

source_folder = './csv_dumps/'
files = os.listdir(f'{source_folder}')


In [196]:
for file in files:
    data = pd.read_csv(f'{source_folder}/{file}')
    data['category'] = file.replace('.csv', '')
    final_file = pd.concat([final_file, data])

In [197]:
final_file.to_csv('csv_final/final_products.csv', index=False)